In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
path = os.getcwd()

In [ ]:
path

In [ ]:
data = pd.read_csv(r'C:\\Users\\ADMIN\\Desktop\\Movie-Recommender/game_info.csv')

In [ ]:
data.head()

In [ ]:
data.drop(columns=['metacritic', 'esrb_rating', 'website', 'slug', 'rating_top', 'tba', 'updated', 'achievements_count', 'game_series_count'], inplace = True)

In [ ]:
data

In [ ]:
data['developers'].value_counts()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data['ratings_count'].sort_values(ascending=False)

In [ ]:
data = data[data['ratings_count'] > 50]

In [ ]:
data

In [ ]:
ratings = data['rating']

In [ ]:
counts = data['ratings_count']

In [ ]:
rating_count_threshold = 50
mean_rating = data['rating'].mean()

In [ ]:
data['weighted_rating'] = ((ratings * counts) + (rating_count_threshold * mean_rating)) / (counts + rating_count_threshold)

In [ ]:
data.sort_values('weighted_rating', ascending = False)

In [ ]:
content_df = data[['id', 'name', 'platforms', 'developers', 'genres', 'publishers']]

In [ ]:
content_df

In [ ]:
content_df.isnull().sum()

In [ ]:
content_df.dropna(inplace=True)

In [ ]:
content_df

In [ ]:
def make_list(obj):
    obj = obj
    tags = []
    devs = obj['developers'].split('||')
    genres = obj['genres'].split('||')
    pubs = obj['publishers'].split('||')

    for tag in devs:
        tags.append(tag)
    for tag in genres:
        tags.append(tag)
    for tag in pubs:
        tags.append(tag)
    return tags

In [ ]:
tags_dict = {}
tags_dict['tags'] = []
for index, row in content_df.iterrows():
    tag = make_list(row)
    tags_dict['tags'].append(tag)

In [ ]:
tags_dict

In [ ]:
for i in range(len(tags_dict['tags'])):
    tags_dict['tags'][i] = list(set(tags_dict['tags'][i]))

In [ ]:
tags_dict

In [ ]:
tags = pd.Series(tags_dict['tags'], name='Tags')

In [ ]:
tags

In [ ]:
content_df = content_df.reset_index()

In [ ]:
content_df = pd.concat([content_df, tags], axis=1)

In [ ]:
content_df

In [34]:
content_df['Tags'] = content_df['Tags'].apply(lambda x: " ".join(x))

In [35]:
content_df['Tags'] = content_df['Tags'].apply(lambda x: x.lower())

In [36]:
content_df.head(10)

,index,id,name,platforms,developers,genres,publishers,Tags
0,13,10001,Serious Sam Fusion 2017 (beta),Linux||macOS||PC,Croteam,Action||Indie,Devolver Digital||Croteam,action croteam indie devolver digital
1,238,10023,Geneshift,Linux||PC,Nik Nak Studios,Action||RPG||Indie,Nik Nak Studios,indie action rpg nik nak studios
2,260,10025,BRAIN / OUT,PC||macOS||Linux,Desertkun,Action||Massively Multiplayer||Indie,Desertkun,desertkun indie action massively multiplayer
3,291,10028,Space Engineers,PC||Xbox One,Keen Software House,Action||Strategy||Simulation||Indie,Keen Software House,indie simulation strategy action keen software...
4,300,10029,Get Even,PC||Xbox One||PlayStation 4,BANDAI NAMCO Entertainment America||The Farm 51,Action||Adventure||Indie,Bandai Namco Entertainment||BANDAI NAMCO Enter...,indie bandai namco entertainment the farm 51 a...
5,322,10031,Battlerite,PC,Stunlock Studios,Action,Stunlock Studios,action stunlock studios
6,359,10035,HITMAN,Linux||macOS||PC||Xbox One||PlayStation 4,Feral Interactive||Square Enix||IO Interactive,Action||Shooter,Square Enix||Feral Interactive||Warner Bros. I...,shooter square enix io interactive feral inter...
7,379,10037,Europa Universalis IV,PC||Linux||macOS,Paradox Development Studio,Strategy,Paradox Interactive,paradox interactive paradox development studio...
8,401,10039,Tyranny,macOS||Linux||PC,Obsidian Entertainment,RPG,Paradox Interactive,paradox interactive rpg obsidian entertainment
9,413,10040,Stellaris,Linux||PC||Nintendo Switch||macOS,Paradox Development Studio,Strategy||Simulation,Paradox Interactive,paradox interactive simulation paradox develop...


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [39]:
vectors = cv.fit_transform(content_df['Tags']).toarray()

In [40]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(2402,))

In [41]:
cv.get_feature_names_out()

array(['10', '1041', '10tons', ..., 'ブロードメディアgc株式会社', '个人', '方块游戏'],
      shape=(2402,), dtype=object)

In [42]:
len(cv.get_feature_names_out())

2402

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
## calculating the distances for each game
similarity = cosine_similarity(vectors)

In [45]:
similarity

array([[1.        , 0.36514837, 0.4       , ..., 0.        , 0.18257419,
        0.51639778],
       [0.36514837, 1.        , 0.36514837, ..., 0.22645541, 0.16666667,
        0.47140452],
       [0.4       , 0.36514837, 1.        , ..., 0.        , 0.18257419,
        0.51639778],
       ...,
       [0.        , 0.22645541, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.18257419, 0.16666667, 0.18257419, ..., 0.        , 1.        ,
        0.23570226],
       [0.51639778, 0.47140452, 0.51639778, ..., 0.        , 0.23570226,
        1.        ]], shape=(3005, 3005))

In [46]:
## main function to get the recommended games
def recommend(game):
    game_index = content_df[content_df['name'] == game].index[0]
    distances = similarity[game_index]
    games_list = sorted(list(enumerate((distances))), reverse=True,key = lambda x: x[1])[1:6]

    for i in games_list:
      print(content_df.iloc[i[0]]['name'])

In [47]:
recommend('Geneshift')

Ziggurat
Salt and Sanctuary
TOXIKK
Divine Divinity
Beyond Divinity


In [48]:
content_df.iloc[39]['name']

'Blackwake'

In [49]:
import pickle

In [50]:
pickle.dump(content_df.to_dict(), open('games_dict(final).pkl', 'wb'))

In [51]:
content_df['name'].values

array(['Serious Sam Fusion 2017 (beta)', 'Geneshift', 'BRAIN / OUT', ...,
       'I Have No Mouth, and I Must Scream', 'Arma 3', 'Nation Red'],
      shape=(3005,), dtype=object)

In [52]:
pickle.dump(similarity, open('similarity(final).pkl', 'wb'))